In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('SampleData.csv',parse_dates=['date'])

In [3]:
#Computing Hourly Data(If required)
data['hourly']=pd.to_datetime(data.date)+pd.to_timedelta(data.hour, unit='h')

## Table 1:Temperature & leaf wetness (LW)

In [4]:
#Creating table1 with AVG Daily Values
table1=data.groupby('date').mean()

In [5]:
#function to compute Table 1:Temperature & leaf wetness (LW)

def compute_t_LW_risk(df):
    if df['TC']<5:
        return 'Low'
    elif df['TC']>=5 and df['TC']<18:
        if df['LW']<14:
            return 'Low'
        else:
            return 'Medium'
    elif df['TC']>=18 and df['TC']<25:
        if df['LW']<6:
            return 'Low'
        elif df['LW']>=6 and df['LW']<12:
            return 'Medium'
        else:
            return 'High'
    elif df['TC']>=25 and df['TC']<30:
        if df['LW']<10:
            return 'Low'
        else:
            return 'Medium'
    else:
        return 'Low'

In [17]:
#Table 1

table1['T_LW_risk']=table1.apply(compute_t_LW_risk, axis=1)
print(table1['T_LW_risk'].value_counts())
table1.head()

Low    127
Name: T_LW_risk, dtype: int64


,month,hour,TC,HUM,LW,PLV2,T_LW_risk,rainfall_risk,cumulative_risk,cumulative_risk_rank,spray
date,,,,,,,,,,,
2021-01-02,2.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-03,3.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-04,4.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-05,5.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-25,1.0,18.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No


## Table 2: Rain effect on model

In [7]:
#Creating table2 with 10 Day AVG Values
table2=table1
#Computing Rolling avg for 10 days
table2['TC'] =   table2['TC'].rolling(10).mean()
table2['HUM'] =  table2['HUM'].rolling(10).mean()
table2['LW'] =   table2['LW'].rolling(10).mean()
table2['PLV2'] = table2['PLV2'].rolling(10).mean()

In [8]:
#Function to compute Table 2: Rain effect on model
def compute_rainfall(df):
    if df['PLV2']>=0.28 and df['PLV2']<3 or df['PLV2']>=25:
        return 'Low'
    elif df['PLV2']>=3 and df['PLV2']<7:
        return 'Medium'
    else:
        return 'High'

In [18]:
#Table 2
table2['rainfall_risk']=table2.apply(compute_rainfall, axis=1)
print(table2['rainfall_risk'].value_counts())
table2.head()

High    127
Name: rainfall_risk, dtype: int64


,month,hour,TC,HUM,LW,PLV2,T_LW_risk,rainfall_risk,cumulative_risk,cumulative_risk_rank,spray
date,,,,,,,,,,,
2021-01-02,2.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-03,3.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-04,4.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-05,5.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-25,1.0,18.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No


## Comulative Risk Calculation 

In [10]:
#Calculating cumulative risk
def cumulative_risk(df):
    if df['T_LW_risk']=='High' or df['rainfall_risk']=='High':
        return 'H'
    elif df['T_LW_risk']=='Medium' or df['rainfall_risk']=='Medium':
        return 'M'
    else:
        return 'L'

In [19]:
#Table with cumulative risk
table2['cumulative_risk']=table2.apply(cumulative_risk,axis=1)

## Spray Condition

In [14]:
#H = 2M
#H=2
#M=1
#spray if > 8 (4H/8M)
table2['cumulative_risk_rank']=table2['cumulative_risk'].apply(lambda x : 2 if x=='H' else (1 if x=='M' else 0))
table2['cumulative_risk_rank']=table2['cumulative_risk_rank'].rolling(10).sum()
table2['cumulative_risk_rank']=table2['cumulative_risk_rank'].fillna(0)

In [15]:
#Spray function
table2['spray']='No'
for i in range(len(table2)):
    label=0
    if (i+1)%10==0:
        if table2['cumulative_risk_rank'][i]>8:
            table2['spray'][i]='Yes'
        else:
            table2['spray'][i]='No'
            label=1
    elif (i+1)%15==0 and label==1:
            table2['spray'][i]='Yes'
    else:
        table2['spray'][i]='No'
            

<ipython-input-15-f2478862113a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2['spray'][i]='No'
<ipython-input-15-f2478862113a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2['spray'][i]='Yes'


In [16]:
table2.head(20)

,month,hour,TC,HUM,LW,PLV2,T_LW_risk,rainfall_risk,cumulative_risk,cumulative_risk_rank,spray
date,,,,,,,,,,,
2021-01-02,2.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-03,3.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-04,4.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-05,5.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-25,1.0,18.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-26,1.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-27,1.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-28,1.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
2021-01-29,1.0,11.5,NaN,NaN,NaN,NaN,Low,High,H,0.0,No
